# [문제]
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory()
- RunnableWithMessageHistory()
- chain 구성

## [문제] 기존 String PromptTemplate을 ChatPromptTemplate으로 변경
- Chain
- prompt template: 직접 작성
                   문서를 참고해서, 사용자 질문에 답변

In [5]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
from langchain_core.chat_history import (
    InMemoryChatMessageHistory,
    BaseChatMessageHistory
)

## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:50])

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 6. chain 구성
chain = prompt | llm

## 7. 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()

# store 임시 메모리리
store = {}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return history

## 9. 사용자 질문 받기
query = input('육아휴직과 관련된 질문하세요 >> ')

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
with_message_history.invoke(
    {'law': law, 'query': query},
    config={'configurable': {'session_id':'abc'}}
)
get_history('abc')


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20
남녀고용평등법 제19조(육아휴직)에 따라, 기본적으로 육아휴직의 기간은 1년 이내로 정해져 있습니다(제19조 제2항). 다만, 특정 조건에 해당하는 근로자는 6개월 추가로 육아휴직을 더 사용할 수 있습니다. 예를 들어, 같은 자녀를 대상으로 부모가 모두 각각 3개월 이상 육아휴직을 사용하거나, 한부모가족지원법 제4조제1호의 부 또는 모, 또는 장애아동의 부모인 경우 등입니다.

즉, 일반적으로는 1년이 최대 기간이지만, 일부 특별한 경우에는 6개월 더 연장하여 최대 1년 6개월까지 사용할 수 있습니다(제19조 제2항, 제19조의4 제1항).

추가로, 구체적인 사용 방법이나 조건은 법률 또는 대통령령에 따라 정해지니, 상세한 사항은 관련 법령 또는 기업의 내부 규정을 참고하시기 바랍니다.

InMemoryChatMessageHistory(messages=[HumanMessage(content='1년만 쓸수있나요?'), AIMessage(content='남녀고용평등법 제19조(육아휴직)에 따라, 기본적으로 육아휴직의 기간은 1년 이내로 정해져 있습니다(제19조 제2항). 다만, 특정 조건에 해당하는 근로자는 6개월 추가로 육아휴직을 더 사용할 수 있습니다. 예를 들어, 같은 자녀를 대상으로 부모가 모두 각각 3개월 이상 육아휴직을 사용하거나, 한부모가족지원법 제4조제1호의 부 또는 모, 또는 장애아동의 부모인 경우 등입니다.\n\n즉, 일반적으로는 1년이 최대 기간이지만, 일부 특별한 경우에는 6개월 더 연장하여 최대 1년 6개월까지 사용할 수 있습니다(제19조 제2항, 제19조의4 제1항).\n\n추가로, 구체적인 사용 방법이나 조건은 법률 또는 대통령령에 따라 정해지니, 상세한 사항은 관련 법령 또는 기업의 내부 규정을 참고하시기 바랍니다.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-7a6fd1fb-7ce0-460c-a6d3-abd795e9abad-0')])